# Part  1: Scraping

In [1]:
!pip3 install splinter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip3 install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip3 install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import Dependencies
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import os
import requests
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd

In [7]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

### Step1: Visit [Mars NASA news site](https://redplanetscience.com) and Scrape the Website for Title and Preview


In [8]:
# TOP NEWS SCRAPE
news_url = "https://redplanetscience.com"
browser.visit(news_url)
html_1 = browser.html
soup_1 = bs(html_1, "html.parser")
latest_news = soup_1.find_all("div", class_="content_title")[1]
latest_news_title = latest_news.get_text()
paragraph = soup_1.find_all("div", class_="article_teaser_body")[0]
latest_news_paragraph = paragraph.text

In [9]:
print(latest_news_title)

New Selfie Shows Curiosity, the Mars Chemist


In [10]:
print(latest_news_paragraph)

Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.


### Step2: Visit [JPL Mars Space webpage](https://spaceimages-mars.com/) and Scrape Featured Image

In [11]:
url = "https://spaceimages-mars.com/"
browser.visit(url)
html_2 = browser.html
soup = bs(html_2, "html.parser")
img = [i.get("src") for i in soup.find_all("img", class_="headerimage fade-in")]
featured_image_url = url + img[0]

In [12]:
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


### Step3: Visit the [Mars Facts webpage](https://galaxyfacts-mars.com) and scrape facts table containing diameter, mass etc

In [13]:
table_url = "https://galaxyfacts-mars.com/"
browser.visit(table_url)
html_3 = browser.html
soup_3 = bs(html_3, "html.parser")
table = soup_3.find_all("table", class_="table")[0]

table_header = [i.text for i in table("th")]
mars_column = [i.text for i in table("span", class_="orange")]
earth_column = [i.text for i in table("span", class_="purple")]

table_df = {"Description": table_header, "Mars": mars_column, "Earth": earth_column}


In [14]:
table_df

{'Description': [' Mars - Earth Comparison',
  'Diameter:',
  'Mass:',
  'Moons:',
  'Distance from Sun:',
  'Length of Year:',
  'Temperature:'],
 'Mars': [' Mars',
  '6,779 km',
  '6.39 × 10^23 kg ',
  '2',
  '227,943,824 km',
  '687 Earth days',
  '-87 to -5 °C'],
 'Earth': [' Earth ',
  '12,742 km',
  '5.97 × 10^24 kg',
  '1',
  '149,598,262 km',
  '365.24 days',
  '\t-88 to 58°C']}

In [15]:
df = pd.DataFrame(table_df)
df.set_index("Description", inplace=True)
df["Earth"] = df["Earth"].str.replace("\t", "")
comparison_table = df.to_html(classes="table table-striped")

In [16]:
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


### Step4: Visit the astrogeology site (https://marshemispheres.com) to obtain high-resolution images for each hemisphere of Mars.

In [17]:
# HEMISPHERE IMAGE SCRAPE
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)

In [18]:
hemisphere_image_urls = []

for i in range(4):
    html = browser.html
    soup = bs(html, "html.parser")
    
    title = soup.find_all("h3")[i].get_text()
    browser.find_by_tag('h3')[i].click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    img_url = soup.find("img", class_="wide-image")["src"]
    
    hemisphere_image_urls.append({
        "title": title,
        "img_url": "https://marshemispheres.com/" + img_url
    })
    browser.back()

In [19]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [20]:
# Append to a list
title1 = hemisphere_image_urls[0]["title"]
image1 = hemisphere_image_urls[0]["img_url"]

title2 = hemisphere_image_urls[1]["title"]
image2 = hemisphere_image_urls[1]["img_url"]

title3 = hemisphere_image_urls[2]["title"]
image3 = hemisphere_image_urls[2]["img_url"]

title4 = hemisphere_image_urls[3]["title"]
image4 = hemisphere_image_urls[3]["img_url"]

In [21]:
# FINAL DICTIONARY FOR MONGO
    
final_mars_data = {
"latest_title": latest_news_title,
"latest_paragraph" : latest_news_paragraph,
"featured_image": featured_image_url,
"html_table": comparison_table,
"hemisphere_scrape": hemisphere_image_urls,
"title1": title1,
"title2": title2,
"title3": title3,
"title4": title4,
"image1": image1,
"image2": image2,
"image3": image3,
"image4": image4,
}



In [22]:
print(final_mars_data)

{'latest_title': 'New Selfie Shows Curiosity, the Mars Chemist', 'latest_paragraph': 'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.', 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg', 'html_table': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</t

In [23]:
df_final = pd.DataFrame(final_mars_data)
df_final

,latest_title,latest_paragraph,featured_image,html_table,hemisphere_scrape,title1,title2,title3,title4,image1,image2,image3,image4
0,"New Selfie Shows Curiosity, the Mars Chemist","Like much of the rest of the world, the Mars r...",https://spaceimages-mars.com/image/featured/ma...,"<table border=""1"" class=""dataframe table table...","{'title': 'Cerberus Hemisphere Enhanced', 'img...",Cerberus Hemisphere Enhanced,Schiaparelli Hemisphere Enhanced,Syrtis Major Hemisphere Enhanced,Valles Marineris Hemisphere Enhanced,https://marshemispheres.com/images/f5e372a36ed...,https://marshemispheres.com/images/3778f7b43bb...,https://marshemispheres.com/images/555e6403a6d...,https://marshemispheres.com/images/b3c7c6c9138...
1,"New Selfie Shows Curiosity, the Mars Chemist","Like much of the rest of the world, the Mars r...",https://spaceimages-mars.com/image/featured/ma...,"<table border=""1"" class=""dataframe table table...","{'title': 'Schiaparelli Hemisphere Enhanced', ...",Cerberus Hemisphere Enhanced,Schiaparelli Hemisphere Enhanced,Syrtis Major Hemisphere Enhanced,Valles Marineris Hemisphere Enhanced,https://marshemispheres.com/images/f5e372a36ed...,https://marshemispheres.com/images/3778f7b43bb...,https://marshemispheres.com/images/555e6403a6d...,https://marshemispheres.com/images/b3c7c6c9138...
2,"New Selfie Shows Curiosity, the Mars Chemist","Like much of the rest of the world, the Mars r...",https://spaceimages-mars.com/image/featured/ma...,"<table border=""1"" class=""dataframe table table...","{'title': 'Syrtis Major Hemisphere Enhanced', ...",Cerberus Hemisphere Enhanced,Schiaparelli Hemisphere Enhanced,Syrtis Major Hemisphere Enhanced,Valles Marineris Hemisphere Enhanced,https://marshemispheres.com/images/f5e372a36ed...,https://marshemispheres.com/images/3778f7b43bb...,https://marshemispheres.com/images/555e6403a6d...,https://marshemispheres.com/images/b3c7c6c9138...
3,"New Selfie Shows Curiosity, the Mars Chemist","Like much of the rest of the world, the Mars r...",https://spaceimages-mars.com/image/featured/ma...,"<table border=""1"" class=""dataframe table table...",{'title': 'Valles Marineris Hemisphere Enhance...,Cerberus Hemisphere Enhanced,Schiaparelli Hemisphere Enhanced,Syrtis Major Hemisphere Enhanced,Valles Marineris Hemisphere Enhanced,https://marshemispheres.com/images/f5e372a36ed...,https://marshemispheres.com/images/3778f7b43bb...,https://marshemispheres.com/images/555e6403a6d...,https://marshemispheres.com/images/b3c7c6c9138...


In [24]:
browser.quit()

# Part 2: MongoDB and Flask Application

In [25]:
def scrape():
    # FEATURED IMAGE SCRAPE
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    url = "https://spaceimages-mars.com/"
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")
    img = [i.get("src") for i in soup.find_all("img", class_="headerimage fade-in")]
    featured_image_url = url + img[0]

    # TOP NEWS SCRAPE
    news_url = "https://mars.nasa.gov/news/"
    browser.visit(news_url)
    html_1 = browser.html
    soup_1 = bs(html_1, "html.parser")
    latest_news = soup_1.find_all("div", class_="content_title")[1]
    latest_news_title = latest_news.get_text()
    paragraph = soup_1.find_all("div", class_="article_teaser_body")[0]
    latest_news_paragraph = paragraph.text
    
    # HEMISPHERE IMAGE SCRAPE
    hemi_url = "https://marshemispheres.com/"
    browser.visit(hemi_url)
    hemisphere_image_urls = []

    for i in range(4):
        html = browser.html
        soup = bs(html, "html.parser")
    
        title = soup.find_all("h3")[i].get_text()
        browser.find_by_tag('h3')[i].click()
    
        html = browser.html
        soup = bs(html, "html.parser")
    
        img_url = soup.find("img", class_="wide-image")["src"]
    
        hemisphere_image_urls.append({
            "title": title,
            "img_url": hemi_url + img_url
        })
        browser.back()
        
    title1 = hemisphere_image_urls[0]["title"]
    image1 = hemisphere_image_urls[0]["img_url"]
    
    title2 = hemisphere_image_urls[1]["title"]
    image2 = hemisphere_image_urls[1]["img_url"]

    title3 = hemisphere_image_urls[2]["title"]
    image3 = hemisphere_image_urls[2]["img_url"]

    title4 = hemisphere_image_urls[3]["title"]
    image4 = hemisphere_image_urls[3]["img_url"]
          

    # MARS FACT TABLE SCRAPE
    table_url = "https://galaxyfacts-mars.com/"
    browser.visit(table_url)
    html_3 = browser.html
    soup_3 = bs(html_3, "html.parser")
    table = soup_3.find_all("table", class_="table")[0]

    table_header = [i.text for i in table("th")]
    mars_column = [i.text for i in table("span", class_="orange")]
    earth_column = [i.text for i in table("span", class_="purple")]

    table_df = {"Description": table_header, "Mars": mars_column, "Earth": earth_column}
    df = pd.DataFrame(table_df)
    df.set_index("Description", inplace=True)
    df["Earth"] = df["Earth"].str.replace("\t", "")
    comparison_table = df.to_html(classes="table table-striped")

    browser.quit()


    # FINAL DICTIONARY FOR MONGO
    
    final_mars_data = {
    "latest_title": latest_news_title,
    "latest_paragraph" : latest_news_paragraph,
    "featured_image": featured_image_url,
    "html_table": comparison_table,
    "hemisphere_scrape": hemisphere_image_urls,
    "title1": title1,
    "title2": title2,
    "title3": title3,
    "title4": title4,
    "image1": image1,
    "image2": image2,
    "image3": image3,
    "image4": image4,

    }

    return final_mars_data